In [2]:
import numpy as np
from os import walk
import re
import requests
from bs4 import BeautifulSoup,Comment
import re
import pandas as pd

In [3]:
FOLDER_PATH = './dataset/foodnetwork/'
test_file = '0a5ae5d3ccf0e0012ccb68716c9af39c.html'

In [4]:
soup_test = BeautifulSoup(open(FOLDER_PATH + test_file, 'rb'), 'html.parser')

In [5]:
title = soup_test.find("meta", property="og:title")['content']
title

'Spicy Couscous'

In [160]:
soup_test.find('ul',class_="kv-ingred-list1").get_text().split('\n')[1:-1]

['2 tablespoons canola oil',
 '2 teaspoons minced garlic',
 '1 teaspoon crushed red pepper flakes',
 '2 cups chicken broth',
 '1 (10-ounce) box couscous',
 'Salt and freshly ground black pepper']

In [161]:
ingredient_list = []
direction_list = []

In [162]:
def normalize_string(string):
    return re.sub(
        r'\s+', ' ',
        string.replace(
            '\xa0', ' ').replace(  # &nbsp;
            '\n', ' ').replace(
            '\t', ' ').strip()
    )

In [163]:
TIME_REGEX = re.compile(
    r'(\D*(?P<hours>\d+)\s*(hours|hrs|hr|h|Hours|H))?(\D*(?P<minutes>\d+)\s*(minutes|mins|min|m|Minutes|M))?')

def get_minutes(element):
    try:
        tstring = element.get_text()
        if '-' in tstring:
            tstring = tstring.split('-')[1]  # sometimes formats are like this: '12-15 minutes'
        matched = TIME_REGEX.search(tstring)

        minutes = int(matched.groupdict().get('minutes') or 0)
        minutes += 60 * int(matched.groupdict().get('hours') or 0)

        return minutes
    except AttributeError:  # if dom_element not found or no matched
        return 0

In [164]:
ingredients = soup_test.findAll('li', {'class': 'ingredient'})
ingredient_list.append({'ingredient':[normalize_string(ingredient.get_text())for ingredient in ingredients],
                        'title': title} ) 
directions = soup_test.findAll('div', class_="instructions")
direction_list.append({'direction':[normalize_string(direction.get_text()) for direction in directions],
                       'title': title} )

In [165]:
ingredient_list

[{'ingredient': ['2 tablespoons canola oil',
   '2 teaspoons minced garlic',
   '1 teaspoon crushed red pepper flakes',
   '2 cups chicken broth',
   '1 (10-ounce) box couscous',
   'Salt and freshly ground black pepper'],
  'title': 'Spicy Couscous'}]

In [166]:
direction_list

[{'direction': ['Heat oil in a medium saucepan over medium-low heat. Saute garlic with red pepper flakes for 2 minutes being careful not to burn garlic. Add the chicken broth, turn up heat to medium-high and bring to a boil. Stir in couscous, cover and remove from the heat. Let stand 5 minutes. Season with salt and pepper and fluff with fork. Transfer to a serving bowl and serve.'],
  'title': 'Spicy Couscous'}]

In [167]:
# Total time
print(soup_test.find('dt',{'class':"head time"}).text)
print(get_minutes(soup_test.find('dd',class_="head duration clrfix")))

Total Time: 
20


In [168]:
# prep time
print(get_minutes(soup_test.find('dd',class_="prepTime clrfix")))

5


In [171]:
# cook time
print(get_minutes(soup_test.find('dd',class_="cookTime clrfix")))

10


In [174]:
# rating and review counts
int(re.findall('(\d+)',soup_test.findAll('span', {'class':"lgbtn-text"})[1].text)[0])

5

In [ ]:
rating_review_list, times_list, ingredient_list, direction_list = ([] for i in range(5))
count=0
for root, _, files in walk(FOLDER_PATH):   #return three list 
    for name in files:
        soup_i = BeautifulSoup(open(FOLDER_PATH + name, 'rb'), 'html.parser')
        if soup_i.findAll('a'):    #make sure there is no blank HTML
            if ('http://www.foodnetwork.com/recipes/' in soup_i.text): # make sure the page is for recipe
                title = soup_i.find("meta", property="og:title")['content']
                rating_review_list.append({'Title': title, 
                                           'Rating': float(re.search(r'(\d*\.\d*)',str(soup_i.find('img', class_="rating")))[0]),
                                           'Reviews': int(re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)[0])\
                                                      if soup_i.findAll('span', {'class':"lgbtn-text"})[1].text else np.nan
                                          })
                times_list.append({'Title': title, 
                                   'Prep Time (min)': get_minutes(soup_i.find('dd',class_="prepTime clrfix"))\
                                                      if soup_i.find('dd',class_="prepTime clrfix") else np.nan,
                                   'Cook Time (min)': get_minutes(soup_i.find('dd',class_="cookTime clrfix"))\
                                                      if soup_i.find('dd',class_="cookTime clrfix") else np.nan,
                                   'Total Time (min)': get_minutes(soup_i.find('dd',class_="head duration clrfix"))\
                                                     if soup_i.find('dd',class_="head duration clrfix") else np.nan
                                  })
                ingredients = soup_i.findAll('li', {'class': 'ingredient'})
                ingredient_list.append({{'Title': title,
                                         'ingredient':[normalize_string(ingredient.get_text())for ingredient in ingredients]}\
                                                      if ingredients else np.nan
                                       }) 
                directions = soup_i..findAll('div', class_="instructions")
                direction_list.append({'Title': title,
                                       'Direction': [normalize_string(direction.get_text()) for direction in directions]\
                                                    if directions else np.nan
                                      })
                print(count, name)
            else:
                print(count, name, 'not recipe pages')
        else:
            print(count, name, 'null pages')
        count+=1